#EXERCISE-2

Que.1

let $x_i=
  \begin{cases}
                                   0 & \text{if student doesn't take Ci course} \\
                                   1 & \text{if student take Ci course} 
  \end{cases}
$ for i =0,1,2,...,10


Our mathematical model for the problem will be:
$$\text{max} \sum_{i=0}^{i=10}a_ix_i , \text{where }a_i \text{ is the credit of course}\ C_i$$
subject to
$$-\sum_{i=0}^{i=10}a_i x_i\leq -30\\ -\sum_{i=0}^{i=10}x_i\leq -5\\ \sum_{i=0}^{i=10}x_i\leq 9\\ x_2+x_7\leq 1\\x_4+x_9\leq1\\x_2+x_8\leq1\\x_{10}-x_0\leq0\\ x_0-x_{10}\leq1\\-x_0-x_1-x_2\leq-1\\x_0+x_1+x_2\leq2\\x_5+x_6\leq1\\x_3+x_6+x_9\leq2 $$

#Que.3

In [43]:
!pip install -q pyomo

In [44]:
from pyomo.environ import *

In [45]:
import numpy as np

In [46]:
model=ConcreteModel()

In [47]:
M=11
N=12

In [48]:
obj_coef=np.array([3,2,6,5,5,4,6,8,8,5,4])
constr_coef_A=np.array([[-3,-2,-6,-5,-5,-4,-6,-8,-8,-5,-4],
                        [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                        [1,1,1,1,1,1,1,1,1,1,1],
                        [0,0,1,0,0,0,0,1,0,0,0],
                        [0,0,0,0,1,0,0,0,0,1,0],
                        [0,0,1,0,0,0,0,0,1,0,0],
                        [-1,0,0,0,0,0,0,0,0,0,1],
                        [1,0,0,0,0,0,0,0,0,0,-1],
                        [-1,-1,-1,0,0,0,0,0,0,0,0],
                        [1,1,1,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,1,1,0,0,0,0],
                        [0,0,0,1,0,0,1,0,0,1,0]
                        ])
constr_rhs_b=np.array([-30,-5,9,1,1,1,0,1,-1,2,1,2])

In [49]:
col_indices=np.arange(M)
row_indices=np.arange(N)

In [50]:
model.x=Var(col_indices,domain=Binary)


In [51]:
model.constraints=ConstraintList()

In [52]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i,j]*model.x[j] for j in col_indices)<=constr_rhs_b[i])

In [53]:
model.objective=Objective(expr=summation(obj_coef,model.x),sense=maximize)

In [54]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  N

In [55]:
!apt-get install -y -qq coinor-cbc

In [56]:
opt_cbc=SolverFactory('cbc')


In [57]:
result=opt_cbc.solve(model)
print(result)



Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03220248222351074
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



**optimal value and values of the decision variables:-**

In [58]:
print("The optimal value is: ",model.objective())
for j in col_indices:
  print(f"x[{j}]=",model.x[j].value)
  

The optimal value is:  41.0
x[0]= 1.0
x[1]= 1.0
x[2]= 0.0
x[3]= 1.0
x[4]= 1.0
x[5]= 0.0
x[6]= 1.0
x[7]= 1.0
x[8]= 1.0
x[9]= 0.0
x[10]= 1.0


#Que.4

modifying the model assuming that the variables are real.

In [59]:
for j in row_indices:
  model.constraints[j+1].deactivate()

In [60]:
model.objective.deactivate()

In [61]:
model.y=Var(col_indices)

In [62]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i,j]*model.y[j] for j in col_indices)<=constr_rhs_b[i])

In [63]:
model.new_objective=Objective(expr=summation(obj_coef,model.y),sense=maximize)

In [64]:
lower_bound=np.array([0,0,0,0,0,0,0,0,0,0,0])
upper_bound=np.array([1,1,1,1,1,1,1,1,1,1,1])


In [65]:
for j in col_indices:
  model.y[j].setlb(lower_bound[j])
  model.y[j].setub(upper_bound[j])

In [66]:
model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

2 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False

In [67]:
result=opt_cbc.solve(model)
print(result)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 12
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.02086615562438965
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



**optimal vlaue and values of decision variables:-**

In [68]:
print("The optimal value is: ",model.new_objective())
for j in col_indices:
  print(f"y[{j}]=",model.y[j].value)

The optimal value is:  41.0
y[0]= 1.0
y[1]= 1.0
y[2]= 0.0
y[3]= 1.0
y[4]= 1.0
y[5]= 0.0
y[6]= 1.0
y[7]= 1.0
y[8]= 1.0
y[9]= 0.0
y[10]= 1.0


#Que.5

In general,  the solution of the MILP can't be obtained by merely rounding the solution of the LP, we need to check the feasibility of the solution of the LP to the original MLIP

here we are getting an optimal solution to the LP which is also feasible for the original MLIP,so here we can get the solution of the MLIP by merely rounding the solution of LP.


#Que.6

The course timing for C11
overlaps with the course timings of C7, C8, C9 so we need to add following constraints in our model.

$x_7+x_{11}\leq1\\ x_8+x_{11}\leq1 \\ x_9+x_{11}\leq1$

After adding these constraints our model will be:

$$\text{max} \sum_{i=0}^{i=11}a_ix_i , \text{where }a_i \text{ is the credit of course}\ C_i$$
subject to
$$-\sum_{i=0}^{i=11}a_i x_i\leq -30\\ -\sum_{i=0}^{i=11}x_i\leq -5\\ \sum_{i=0}^{i=11}x_i\leq 9\\ x_2+x_7\leq 1\\x_4+x_9\leq1\\x_2+x_8\leq1\\x_{10}-x_0\leq0\\ x_0-x_{10}\leq1\\-x_0-x_1-x_2\leq-1\\x_0+x_1+x_2\leq2\\x_5+x_6\leq1\\x_3+x_6+x_9\leq2\\x_7+x_{11}\leq1\\ x_8+x_{11}\leq1 \\ x_9+x_{11}\leq1 $$




#Que.7

In [69]:
constr_new_coef_A=np.array([[-3,-2,-6,-5,-5,-4,-6,-8,-8,-5,-4,-9],
                        [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0],
                        [1,1,1,1,1,1,1,1,1,1,1,0],
                        [0,0,1,0,0,0,0,1,0,0,0,0],
                        [0,0,0,0,1,0,0,0,0,1,0,0],
                        [0,0,1,0,0,0,0,0,1,0,0,0],
                        [-1,0,0,0,0,0,0,0,0,0,1,0],
                        [1,0,0,0,0,0,0,0,0,0,-1,0],
                        [-1,-1,-1,0,0,0,0,0,0,0,0,0],
                        [1,1,1,0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,1,1,0,0,0,0,0],
                        [0,0,0,1,0,0,1,0,0,1,0,0],
                        [0,0,0,0,0,0,0,1,0,0,0,1],
                        [0,0,0,0,0,0,0,0,1,0,0,1],
                        [0,0,0,0,0,0,0,0,0,1,0,1]
                        ])
obj_new_coef=np.array([3,2,6,5,5,4,6,8,8,5,4,9])
N1=12
M1=15
col_new_indices=np.arange(N1)
row_new_indices=np.arange(M1)
modified_model=ConcreteModel()
modified_model.x=Var(col_new_indices,domain=Binary)
constr_new_rhs_b=np.array([-30,-5,9,1,1,1,0,1,-1,2,1,2,1,1,1])


In [70]:
modified_model.objective=Objective(expr=summation(obj_new_coef,modified_model.x),sense=maximize)

In [71]:
modified_model.constraints=ConstraintList()

In [72]:
for i in row_new_indices:
  modified_model.constraints.add(sum(constr_new_coef_A[i,j]*modified_model.x[j] for j in col_new_indices)<=constr_new_rhs_b[i])

In [73]:
modified_model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   15 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
        

In [74]:
result=opt_cbc.solve(modified_model)
print(result)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 12
  Number of binary variables: 12
  Number of integer variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.019463062286376953
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



**optimal value and values of the decision variables:-** 

In [75]:
print("The optimal value is: ",modified_model.objective())
for j in col_new_indices:
  print(f"x[{j}]=",modified_model.x[j].value)

The optimal value is:  41.0
x[0]= 1.0
x[1]= 1.0
x[2]= 0.0
x[3]= 1.0
x[4]= 1.0
x[5]= 0.0
x[6]= 1.0
x[7]= 1.0
x[8]= 1.0
x[9]= 0.0
x[10]= 1.0
x[11]= 0.0


#Que.8

In [76]:
for j in row_new_indices:
  modified_model.constraints[j+1].deactivate()

In [77]:
modified_model.y=Var(col_new_indices)

In [78]:
modified_model.objective.deactivate()

In [79]:
modified_model.new_objective=Objective(expr=summation(obj_new_coef,modified_model.y),sense=maximize)

In [80]:
for i in row_new_indices:
  modified_model.constraints.add(sum(constr_new_coef_A[i,j]*modified_model.y[j] for j in col_new_indices)<=constr_new_rhs_b[i])

In [81]:
new_lower_bound=np.array([0,0,0,0,0,0,0,0,0,0,0,0])
new_upper_bound=np.array([1,1,1,1,1,1,1,1,1,1,1,1])

In [82]:
for j in col_new_indices:
  modified_model.y[j].setlb(new_lower_bound[j])
  modified_model.y[j].setub(new_upper_bound[j])

In [83]:
modified_model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   30 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

2 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0

In [84]:
result=opt_cbc.solve(modified_model)
print(result)
print("The optimal value is: ",modified_model.new_objective())
for j in col_new_indices:
  print(f"y[{j}]=",modified_model.y[j].value)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 13
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 12
  Error rc: 0
  Time: 0.03218245506286621
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

The optimal value is:  41.0
y[0]= 1.0
y[1]= 1.0
y[2]= 0.0
y[3]= 1.0
y[4]= 1.0
y[5]= 0.0
y[6]= 1.0
y[7]= 1.0
y[8]= 1.0
y[9]= 0.0
y[10]= 1.0
y[11]= 0.0


Here we are getting same solution of modified LP and the Modified MILP,so here the solution of the modified MILP can be obtained by merely rounding the solution of the
corresponding modified LP